In [ ]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import calendar
import datetime
from multiprocessing.pool import ThreadPool

In [ ]:
stock_url = "https://www.bseindia.com/markets/equity/EQReports/StockPrcHistori.aspx?flag=0"

In [ ]:
path = os.path.join(os.getcwd(),"Data\Stock")
print(path)

def download_stocks(security_id):

    # set from date
    def set_from_date(d,m,y):
        from_date = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_txtFromDate"]')
        from_date.clear()
        from_date.click()
        year = driver.find_element_by_xpath('/html/body/div[1]/div/div/select[2]') 
        year = Select(year)
        while year.options[0].text > y:
            year.select_by_visible_text(year.options[0].text) 
            year = driver.find_element_by_xpath('/html/body/div[1]/div/div/select[2]') 
            year = Select(year)

        year.select_by_visible_text(y) 

        month = driver.find_element_by_xpath('/html/body/div[1]/div/div/select[1]') 
        month = Select(month)
        month.select_by_visible_text(m)  

        days=driver.find_element_by_xpath("//table/tbody/tr/td/a[text()="+str(d)+"]")
        days.click()

    # set to date
    def set_to_date(d,m,y):
        to_date = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_txtToDate"]')
        to_date.clear()
        to_date.click()
        year = driver.find_element_by_xpath('/html/body/div[1]/div/div/select[2]') 
        year = Select(year)
        while year.options[0].text > y:
            year.select_by_visible_text(year.options[0].text) 
            year = driver.find_element_by_xpath('/html/body/div[1]/div/div/select[2]') 
            year = Select(year)

        year.select_by_visible_text(y) 

        month = driver.find_element_by_xpath('/html/body/div[1]/div/div/select[1]') 
        month = Select(month)
        month.select_by_visible_text(m)  

        days=driver.find_element_by_xpath("//table/tbody/tr/td/a[text()="+str(d)+"]")
        days.click()
    

    def set_security_id(security):
        element = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_smartSearch"]')
        element.clear()
        element.send_keys(security)
        element.send_keys(Keys.ENTER)

    def download():
        submit = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_btnSubmit"]')
        submit.click()
        time.sleep(1)
        driver.find_element_by_xpath("/html/body/form/div[4]/div/div/div[1]/div/div[2]/div/div[1]/div[2]/span/a/i").click()
        time.sleep(3)
        driver.quit()

    def create_driver():
        chromeOptions = webdriver.ChromeOptions()
        # chromeOptions.add_argument("--headless")
        chromeOptions.add_experimental_option("prefs",{"download.default_directory":path})
        driver = webdriver.Chrome(ChromeDriverManager().install(), options = chromeOptions)
        return driver

    if os.path.exists(os.path.join(os.getcwd(),"Data\Stock\\"+str(security_id)+".csv")):
        driver = create_driver()
        driver.get(stock_url)
        old_df = pd.read_csv(os.path.join(os.getcwd(),"Data\Stock\\"+str(security_id)+".csv"))
        old_df["Date"] = pd.to_datetime(old_df["Date"])
        last = old_df["Date"].head(1)[0]

        set_security_id(str(security_id))

        set_from_date(last.day,calendar.month_abbr[last.month],str(last.year))
        today = datetime.date.today()
        set_to_date(today.day,calendar.month_abbr[today.month],str(today.year))
        download()
        new_df = pd.read_csv(os.path.join(os.getcwd(),"Data\Stock\\"+str(security_id)+" (1).csv"))
        new_df["Date"] = pd.to_datetime(new_df["Date"])
        res = new_df.append(old_df)
        res.to_csv(os.path.join(os.getcwd(),"Data\Stock\\"+str(security_id)+".csv"),index=None)
        os.remove(os.path.join(os.getcwd(),"Data\Stock\\"+str(security_id)+" (1).csv"))

    else:
        driver = create_driver()
        driver.get(stock_url)
        set_security_id(str(security_id))
        set_from_date("01","Jan","2000")
        today = datetime.date.today()
        set_to_date(today.day,calendar.month_abbr[today.month],str(today.year))
        download()


In [ ]:
# download_stocks(500002)

In [ ]:
equity = pd.read_csv(os.path.join(os.getcwd(),"Data\Equity.csv"))
security_ids = equity["Security Code"].values

In [ ]:
for i in security_ids[:8]:
    try:
        download_stocks(i)
    except Exception as e:
        print(e)